In [1]:
!pip install numba

MATRIX-MATRIX MULTIPLICATION:

In [6]:
import numpy as np
from numba import cuda, njit, float32
import time

# Constants for the number of threads per block in the GPU kernel
THREADS_PER_BLOCK_ROW = 32
THREADS_PER_BLOCK_COL = 32

@cuda.jit
def matrix_multiply_gpu(A, B, C):
    """
    GPU kernel for matrix multiplication using CUDA.

    Parameters:
    - A, B: Input matrices
    - C: Output matrix
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
      value = 0.0
      # Iterate over the shared dimension of A and B
      for k in range(A.shape[1]):
        value += A[row, k] * B[k, col]
      C[row, col] = value

@njit
def naive_matrix_multiply_cpu(A, B, C):
    """
    Naive CPU implementation for matrix multiplication.

    Parameters:
    - A, B: Input matrices
    - C: Output matrix
    """
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            value = 0.0
            # Iterate over the shared dimension of A and B
            for k in range(A.shape[1]):
                value += A[i, k] * B[k, j]
            C[i, j] = value

def generate_random_square_matrix(size):
    """
    Generate a random square matrix of the specified size.

    Parameter:
    - size: Size of the square matrix

    Returns:
    - Random square matrix
    """
    return (np.random.rand(size, size) * 1000).astype(np.float32)

def check_equal(A1, A2):
    """
    Check if two matrices are equal within a specified tolerance.

    Parameters:
    - A1, A2: Matrices to compare

    Returns:
    - True if matrices are equal, False otherwise
    """
    return np.allclose(A1, A2, rtol=1e-5)

def main():
    # Set the matrix size (original size, not power of two)
    matrix_size = 10

    # Generate random square matrices A and B
    A = generate_random_square_matrix(1 << matrix_size)
    B = generate_random_square_matrix(1 << matrix_size)

    # Initialize output matrices C and C_cpu
    C = np.zeros_like(A)
    C_cpu = np.zeros_like(A)

    # defice A
    d_A = cuda.to_device(A)
    d_B = cuda.to_device(B)
    d_C = cuda.device_array_like(C)

    block_dim = (THREADS_PER_BLOCK_ROW, THREADS_PER_BLOCK_COL)

    grid_dim = (A.shape[0] // block_dim[0], A.shape[1] // block_dim[1])

    print("Matrix size: ", A.shape)
    print("Tgread block dimensions: ", block_dim)
    print("Grid dimensions: ", grid_dim)

    ### ------------------- TO DO 1------------------- ###

    # Perform matrix multiplication on the GPU
    start_time = time.time()
    matrix_multiply_gpu[grid_dim, block_dim](d_A, d_B, d_C)
    cuda.synchronize()

    ### ------------------- TO DO 2------------------- ###
    gpu_time = time.time() - start_time

    ### ------------------- TO DO 3------------------- ###
    d_C.copy_to_host(C)

    # Perform matrix multiplication on the CPU
    start_time = time.time()
    naive_matrix_multiply_cpu(A, B, C_cpu)
    cpu_time = time.time() - start_time

    # Check if the CPU and GPU results are equal
    #if check_equal(C, C_cpu):
    #    print("\nCPU and GPU results are equal")
    #else:
    #    print("\nError! CPU and GPU results are different")

    # Print the execution times
    print(f"GPU Time: {gpu_time:.6f} seconds")
    print(f"CPU Time (naive): {cpu_time:.6f} seconds")

if __name__ == "__main__":
    main()

Matrix size:  (1024, 1024)
Tgread block dimensions:  (32, 32)
Grid dimensions:  (32, 32)
GPU Time: 0.329163 seconds
CPU Time (naive): 3.723518 seconds


NORMALIZATION OF VECTORS:

In [8]:
import numpy as np
from numba import cuda, float32
import math
import time

# Constants for the number of threads per block in the GPU kernel
THREADS_PER_BLOCK = 128

@cuda.jit
def vector_normalize_gpu(input_array, output_array):
    """
    GPU kernel for vector normalization using CUDA.

    Parameters:
    - input_array: Input vector
    - output_array: Normalized vector (output)
    """
    ### ------------------- TO DO ------------------- ###

    # Shared
    s_squared = cuda.shared.array(THREADS_PER_BLOCK, float32)

    # Global thread index
    idx = cuda.grid(1)

    # Local thread index
    local_idx = cuda.threadIdx.x

    # Initialize the shared memory to zero
    s_squared[local_idx] = 0.0

    # Synchronize threads befeore startinv computation
    cuda.syncthreads()

    # Compute the sum of square in each block
    for i in range(local_idx, input_array.shape[0], THREADS_PER_BLOCK):
      s_squared[local_idx] += input_array[i] ** 2

    # Synchronize threads before starting computation
    cuda.syncthreads()

    # Perform parallel reduction to find the total sum
    i = THREADS_PER_BLOCK // 2
    while i > 0:
      if local_idx < i:
        s_squared[local_idx] += s_squared[local_idx + i]
      cuda.synctreads()
      i //= 2

    # Normalize the vector using the total sum of square
    if idx < output_array.shape[0]:
      output_array[idx] = input_array[idx] / math.sqrt(s_squared[0])



def naive_vector_normalize_cpu(input_vector, output_vector):
    """
    Naive CPU implementation for vector normalization.

    Parameters:
    - input_vector: Input vector
    - output_vector: Normalized vector (output)
    """
    sum_of_squares = 0.0

    # Compute the sum of squares
    for value in input_vector:
        sum_of_squares += value ** 2

    # Normalize the vector using the total sum of squares
    for i in range(len(input_vector)):
        output_vector[i] = input_vector[i] / math.sqrt(sum_of_squares)

def generate_random_vector(size):
    """
    Generate a random vector of the specified size.

    Parameter:
    - size: Size of the vector

    Returns:
    - Random vector
    """
    return (np.random.rand(size) * 1000).astype(np.float32)

def check_equal(A1, A2):
    """
    Check if two vectors are equal within a specified tolerance.

    Parameters:
    - A1, A2: Vectors to compare

    Returns:
    - True if vectors are equal, False otherwise
    """
    return np.allclose(A1, A2, rtol=1e-5)

def main():
    # Set the vector size
    vector_size = 500000

    # Generate a random vector
    input_vector = generate_random_vector(vector_size)

    # Initialize output vectors
    output_vector_gpu = np.zeros_like(input_vector)
    output_vector_cpu = np.zeros_like(input_vector)

    ### ------------------- TO DO 1------------------- ###
    d_input_vector = cuda.to_device(input_vector)
    d_output_vector_gpu = cuda.device_array_like(output_vector_gpu)

    block_dim = THREADS_PER_BLOCK
    grid_dim = (vector_size + block_dim - 1) // block_dim



    # Perform vector normalization on the GPU
    start_time = time.time()
    ### ------------------- TO DO 2------------------- ###
    vector_normalize_gpu[grid_dim, block_dim](d_input_vector, d_output_vector_gpu)
    cuda.synchronize()
    gpu_time = time.time() - start_time

    ### ------------------- TO DO 3------------------- ###
    d_output_vector_gpu.copy_to_host(output_vector_gpu)

    # Perform vector normalization on the CPU (naive implementation)
    start_time = time.time()
    naive_vector_normalize_cpu(input_vector, output_vector_cpu)
    cpu_time = time.time() - start_time

    # Check if the CPU and GPU results are equal
    if check_equal(output_vector_gpu, output_vector_cpu):
        print("CPU and GPU results are equal\n")
    else:
        print("Error! CPU and GPU results are different\n")

    # Print the execution times
    print(f"GPU Time: {gpu_time:.6f} seconds")
    print(f"CPU Time (naive): {cpu_time:.6f} seconds")

if __name__ == "__main__":
    main()

<ipython-input-8-915df61afa2b>:47: NumbaPendingDeprecationWarning: Code using Numba extension API maybe depending on 'old_style' error-capturing, which is deprecated and will be replaced by 'new_style' in a future release. See details at https://numba.readthedocs.io/en/latest/reference/deprecation.html#deprecation-of-old-style-numba-captured-errors
Exception origin:
  File "/usr/local/lib/python3.10/dist-packages/numba/core/typing/context.py", line 348, in resolve_module_constants
    attrval = getattr(typ.pymod, attr)

  cuda.synctreads()


TypingError: ignored